In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='x8C5DzUSZxEnddIA5h9fXfq8STLXaC13eak3-3xUcQ2y',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'ibmcloud-donotdelete-pr-cqgdlsivcsjmhu'
object_key = 'conversation engine for deaf and dumb.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [3]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [4]:
ls

Dataset/


Image Preprocessing

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
test_datagen = ImageDataGenerator(rescale=1/255)

In [7]:
x_train = train_datagen.flow_from_directory('Dataset/training_set',target_size=(64,64), batch_size=300,class_mode='categorical', color_mode = "grayscale")

Found 15750 images belonging to 9 classes.


In [8]:
x_test = test_datagen.flow_from_directory('Dataset/test_set',target_size=(64,64), batch_size=300,class_mode='categorical', color_mode = "grayscale")

Found 2250 images belonging to 9 classes.


In [9]:
x_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

Model Building

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten

In [11]:
model=Sequential()

In [12]:
model.add(Convolution2D(32, (3,3), input_shape=(64,64,1), activation='relu'))

In [13]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
model.add(Flatten())

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.add(Dense (units=512, activation='relu'))

In [17]:
model.add(Dense (units=9, activation='softmax'))

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
len(x_train)

53

In [20]:
model.fit_generator(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40)

/tmp/wsuser/ipykernel_164/559462756.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train, steps_per_epoch=24, epochs=10, validation_data=x_test, validation_steps=40)


Epoch 1/10
24/24 [==============================] - 69s 3s/step - loss: 1.5135 - accuracy: 0.6124 - val_loss: 0.4504 - val_accuracy: 0.8964
Epoch 2/10
24/24 [==============================] - 63s 3s/step - loss: 0.3472 - accuracy: 0.9046
Epoch 3/10
24/24 [==============================] - 62s 3s/step - loss: 0.1718 - accuracy: 0.9547
Epoch 4/10
24/24 [==============================] - 61s 3s/step - loss: 0.1074 - accuracy: 0.9742
Epoch 5/10
24/24 [==============================] - 62s 3s/step - loss: 0.0749 - accuracy: 0.9815
Epoch 6/10
24/24 [==============================] - 63s 3s/step - loss: 0.0586 - accuracy: 0.9868
Epoch 7/10
24/24 [==============================] - 62s 3s/step - loss: 0.0381 - accuracy: 0.9906
Epoch 8/10
24/24 [==============================] - 61s 3s/step - loss: 0.0334 - accuracy: 0.9915
Epoch 9/10
24/24 [==============================] - 63s 3s/step - loss: 0.0294 - accuracy: 0.9925
Epoch 10/10
24/24 [==============================] - 61s 3s/step - loss: 0.0

In [21]:
model.save('conversation deaf and dumb.h5')

ibm deployment

In [22]:
!tar  -zcvf realtime.tgz 'conversation deaf and dumb.h5'

conversation deaf and dumb.h5


In [23]:
!pip install ibm_watson_machine_learning


In [24]:
from ibm_watson_machine_learning import APIClient

In [27]:
wml_credentials={
    "url":"https://eu-de.ml.cloud.ibm.com",
    "apikey":"ufX4oRE2H8ZkKScsbk6pWRicqla_leNHSh0qL6wdkOgF"
}

In [28]:
client = APIClient(wml_credentials)

In [29]:
client

In [30]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -------------  ------------------------
ID                                    NAME           CREATED
e19d4427-3814-4240-b5bb-1eed3cf5df1a  communication  2022-11-16T16:08:30.386Z
------------------------------------  -------------  ------------------------


In [31]:
space_id = "e19d4427-3814-4240-b5bb-1eed3cf5df1a"

In [32]:
client.set.default_space(space_id)

'SUCCESS'

In [34]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [39]:
software_space_uid = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")

In [40]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [42]:
model_details = client.repository.store_model(model='realtime.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"cnn model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [43]:
model_id = client.repository.get_model_id(model_details)

In [44]:
model_id

'50325158-1f40-47e7-ae49-b8dc9c4fe703'

In [36]:
from keras.models import load_model 
import numpy as np
import cv2

In [37]:
model=load_model('conversation deaf and dumb.h5')

In [67]:
from skimage.transform import resize
def detect(frame):
      img = resize(frame, (64,64,1))
      img = np.expand_dims(img,axis=0) 
      if(np.max(img)>1):
        img=img/255.0
      prediction = model.predict(img)
      print(prediction)
      prediction = model.predict_classes(img)
      print(prediction)

In [69]:
frame=cv2.imread(r"/content/drive/MyDrive/Dataset/test_set/D/10.png")
data = detect(frame)

1/1 [==============================] - 0s 31ms/step
[[3.0667040e-09 3.7450640e-08 5.0424187e-12 9.9998152e-01 2.2859432e-08
  2.1197555e-09 1.1723517e-09 4.2331633e-12 1.8421955e-05]]


AttributeError: ignored